In [1]:
# Perform initial import of the needed libraries

import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import sklearn
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from IPython.display import display

In [2]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split

In [3]:
# Read the CSV and perform basic data cleaning
file_path = Path('./Resources/fights_fighters_records_v2.csv')
fnf_df = pd.read_csv(file_path)[:-2]

In [4]:
fnf_df.columns

Index(['fight_id', 'winner', 'title_bout', 'weight_class', 'no_of_rounds',
       'r_fighter', 'r_fighter_id', 'r_fighter_stance', 'r_fighter_age',
       'b_fighter', 'b_fighter_id', 'b_fighter_stance', 'b_fighter_age',
       'date', 'referee', 'referee_id', 'b_fighter_height', 'r_fighter_height',
       'b_fighter_reach', 'r_fighter_reach', 'b_fighter_weight',
       'r_fighter_weight', 'records_id', 'fight_id-2', 'b_current_lose_streak',
       'b_current_win_streak', 'b_draw', 'b_longest_win_streak', 'b_losses',
       'b_total_title_bouts', 'b_win_by_decision_majority',
       'b_win_by_decision_split', 'b_win_by_decision_unanimous',
       'b_win_by_ko_tko', 'b_win_by_submission',
       'b_win_by_tko_doctor_stoppage', 'b_wins', 'b_total_rounds_fought',
       'b_total_time_fought_seconds', 'r_current_lose_streak',
       'r_current_win_streak', 'r_draw', 'r_longest_win_streak', 'r_losses',
       'r_total_rounds_fought', 'r_total_time_fought_seconds',
       'r_total_title_bout

In [5]:
columns = (['winner', 'title_bout', 'weight_class', 'no_of_rounds', 'r_fighter_stance', 'b_fighter_stance', 'b_fighter_height', 'r_fighter_height', 'b_fighter_reach',
       'r_fighter_reach', 'b_fighter_weight', 'r_fighter_weight', 'b_fighter_age', 'r_fighter_age'])

target = ["winner"]

In [6]:
fnf_df = fnf_df.loc[:, columns].copy()

In [7]:
fnf_df = fnf_df[fnf_df.weight_class != "Women's Bantamweight"]
fnf_df = fnf_df[fnf_df.weight_class != "Women's Featherweight"]
fnf_df = fnf_df[fnf_df.weight_class != "Women's Flyweight"]
fnf_df = fnf_df[fnf_df.weight_class != "Women's Strawweight"]
fnf_df.head(50)

,winner,title_bout,weight_class,no_of_rounds,r_fighter_stance,b_fighter_stance,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight,b_fighter_age,r_fighter_age
0,Red,TRUE,Catch Weight,1,Southpaw,Orthodox,195.58,185.42,NaN,NaN,216.0,175.0,34.0,26.0
1,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,182.88,NaN,205.0,175.0,29.0,26.0
2,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,NaN,NaN,196.0,175.0,30.0,26.0
3,Red,FALSE,Open Weight,1,Southpaw,NaN,187.96,180.34,NaN,NaN,185.0,190.0,NaN,24.0
4,Red,FALSE,Open Weight,1,Orthodox,Orthodox,193.04,195.58,NaN,NaN,275.0,216.0,NaN,34.0
5,Red,FALSE,Open Weight,1,Orthodox,Orthodox,182.88,195.58,NaN,NaN,430.0,216.0,24.0,34.0
6,Red,FALSE,Open Weight,1,Orthodox,Orthodox,187.96,185.42,NaN,182.88,225.0,205.0,30.0,29.0
7,Red,FALSE,Open Weight,1,Orthodox,Orthodox,195.58,193.04,NaN,NaN,250.0,275.0,NaN,NaN
8,Red,FALSE,Open Weight,1,Southpaw,Southpaw,190.50,185.42,NaN,NaN,260.0,175.0,NaN,27.0
9,Red,FALSE,Open Weight,1,Southpaw,Southpaw,180.34,185.42,NaN,NaN,190.0,175.0,24.0,27.0


In [8]:
# for row in fnf_df.weight_class:
#     if fnf_df["weight_class"] == "Women's Bantamweight":
#         print ("found one")a

In [9]:
fnf_df.isnull().sum()

winner                0
title_bout            0
weight_class          0
no_of_rounds          0
r_fighter_stance     90
b_fighter_stance    118
b_fighter_height      8
r_fighter_height      4
b_fighter_reach     662
r_fighter_reach     312
b_fighter_weight      6
r_fighter_weight      3
b_fighter_age       172
r_fighter_age        64
dtype: int64

In [10]:
fnf_df['r_fighter_stance'].fillna("unlisted", inplace = True)
fnf_df['b_fighter_stance'].fillna("unlisted", inplace = True)
fnf_df['r_fighter_height'].fillna(value=fnf_df['r_fighter_height'].mean(), inplace = True)
fnf_df['b_fighter_height'].fillna(value=fnf_df['b_fighter_height'].mean(), inplace = True)
fnf_df['b_fighter_reach'].fillna(value=fnf_df['b_fighter_reach'].mean(), inplace = True)
fnf_df['r_fighter_reach'].fillna(value=fnf_df['r_fighter_reach'].mean(), inplace = True)
fnf_df['b_fighter_weight'].fillna(value=fnf_df['b_fighter_weight'].mean(), inplace = True)
fnf_df['r_fighter_weight'].fillna(value=fnf_df['r_fighter_weight'].mean(), inplace = True)
fnf_df['b_fighter_age'].fillna(value=fnf_df['b_fighter_age'].mean(), inplace = True)
fnf_df['r_fighter_age'].fillna(value=fnf_df['r_fighter_age'].mean(), inplace = True)

In [11]:
fnf_df.isnull().sum()

winner              0
title_bout          0
weight_class        0
no_of_rounds        0
r_fighter_stance    0
b_fighter_stance    0
b_fighter_height    0
r_fighter_height    0
b_fighter_reach     0
r_fighter_reach     0
b_fighter_weight    0
r_fighter_weight    0
b_fighter_age       0
r_fighter_age       0
dtype: int64

In [12]:
fnf_df

,winner,title_bout,weight_class,no_of_rounds,r_fighter_stance,b_fighter_stance,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight,b_fighter_age,r_fighter_age
0,Red,TRUE,Catch Weight,1,Southpaw,Orthodox,195.58,185.42,184.512378,184.798074,216.0,175.0,34.000000,26.0
1,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,182.880000,184.798074,205.0,175.0,29.000000,26.0
2,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,184.512378,184.798074,196.0,175.0,30.000000,26.0
3,Red,FALSE,Open Weight,1,Southpaw,unlisted,187.96,180.34,184.512378,184.798074,185.0,190.0,29.179944,24.0
4,Red,FALSE,Open Weight,1,Orthodox,Orthodox,193.04,195.58,184.512378,184.798074,275.0,216.0,29.179944,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5133,Blue,FALSE,Featherweight,3,Orthodox,Orthodox,180.34,172.72,182.880000,180.340000,145.0,145.0,31.000000,37.0
5134,Red,FALSE,Bantamweight,3,Orthodox,Orthodox,167.64,170.18,165.100000,180.340000,135.0,135.0,32.000000,29.0
5135,Red,TRUE,Bantamweight,5,Orthodox,Orthodox,167.64,162.56,170.180000,162.560000,135.0,135.0,31.000000,32.0
5136,Blue,FALSE,Bantamweight,3,Orthodox,Switch,170.18,162.56,170.180000,172.720000,135.0,135.0,26.000000,29.0


In [13]:
# Reset the indexes and remove the original index
fnf_df = fnf_df.reset_index(drop=True)

In [14]:
fnf_df.dtypes

winner               object
title_bout           object
weight_class         object
no_of_rounds          int64
r_fighter_stance     object
b_fighter_stance     object
b_fighter_height    float64
r_fighter_height    float64
b_fighter_reach     float64
r_fighter_reach     float64
b_fighter_weight    float64
r_fighter_weight    float64
b_fighter_age       float64
r_fighter_age       float64
dtype: object

In [15]:
# Generate our categorical variable lists
app_cat = fnf_df.dtypes[fnf_df.dtypes == "object"].index.tolist()

In [16]:
app_cat = [
 'title_bout',
 'weight_class',
 'r_fighter_stance',
 'b_fighter_stance']

In [17]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(fnf_df[app_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(app_cat)
encode_df.head()

,title_bout_FALSE,title_bout_TRUE,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,...,r_fighter_stance_Sideways,r_fighter_stance_Southpaw,r_fighter_stance_Switch,r_fighter_stance_unlisted,b_fighter_stance_Open Stance,b_fighter_stance_Orthodox,b_fighter_stance_Sideways,b_fighter_stance_Southpaw,b_fighter_stance_Switch,b_fighter_stance_unlisted
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [18]:
pd.options.display.max_columns = None
display(encode_df)

,title_bout_FALSE,title_bout_TRUE,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,r_fighter_stance_Open Stance,r_fighter_stance_Orthodox,r_fighter_stance_Sideways,r_fighter_stance_Southpaw,r_fighter_stance_Switch,r_fighter_stance_unlisted,b_fighter_stance_Open Stance,b_fighter_stance_Orthodox,b_fighter_stance_Sideways,b_fighter_stance_Southpaw,b_fighter_stance_Switch,b_fighter_stance_unlisted
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4824,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4825,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4826,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4827,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [19]:
fnf_df

,winner,title_bout,weight_class,no_of_rounds,r_fighter_stance,b_fighter_stance,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight,b_fighter_age,r_fighter_age
0,Red,TRUE,Catch Weight,1,Southpaw,Orthodox,195.58,185.42,184.512378,184.798074,216.0,175.0,34.000000,26.0
1,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,182.880000,184.798074,205.0,175.0,29.000000,26.0
2,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,184.512378,184.798074,196.0,175.0,30.000000,26.0
3,Red,FALSE,Open Weight,1,Southpaw,unlisted,187.96,180.34,184.512378,184.798074,185.0,190.0,29.179944,24.0
4,Red,FALSE,Open Weight,1,Orthodox,Orthodox,193.04,195.58,184.512378,184.798074,275.0,216.0,29.179944,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4824,Blue,FALSE,Featherweight,3,Orthodox,Orthodox,180.34,172.72,182.880000,180.340000,145.0,145.0,31.000000,37.0
4825,Red,FALSE,Bantamweight,3,Orthodox,Orthodox,167.64,170.18,165.100000,180.340000,135.0,135.0,32.000000,29.0
4826,Red,TRUE,Bantamweight,5,Orthodox,Orthodox,167.64,162.56,170.180000,162.560000,135.0,135.0,31.000000,32.0
4827,Blue,FALSE,Bantamweight,3,Orthodox,Switch,170.18,162.56,170.180000,172.720000,135.0,135.0,26.000000,29.0


In [20]:
encode_df.columns

Index(['title_bout_FALSE', 'title_bout_TRUE ', 'weight_class_Bantamweight',
       'weight_class_Catch Weight', 'weight_class_Featherweight',
       'weight_class_Flyweight', 'weight_class_Heavyweight',
       'weight_class_Light Heavyweight', 'weight_class_Lightweight',
       'weight_class_Middleweight', 'weight_class_Open Weight',
       'weight_class_Welterweight', 'r_fighter_stance_Open Stance',
       'r_fighter_stance_Orthodox', 'r_fighter_stance_Sideways',
       'r_fighter_stance_Southpaw', 'r_fighter_stance_Switch',
       'r_fighter_stance_unlisted', 'b_fighter_stance_Open Stance',
       'b_fighter_stance_Orthodox', 'b_fighter_stance_Sideways',
       'b_fighter_stance_Southpaw', 'b_fighter_stance_Switch',
       'b_fighter_stance_unlisted'],
      dtype='object')

In [21]:
fnf_df_num = pd.concat([fnf_df,encode_df],axis=1)

In [22]:
fnf_df_num

,winner,title_bout,weight_class,no_of_rounds,r_fighter_stance,b_fighter_stance,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight,b_fighter_age,r_fighter_age,title_bout_FALSE,title_bout_TRUE,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,r_fighter_stance_Open Stance,r_fighter_stance_Orthodox,r_fighter_stance_Sideways,r_fighter_stance_Southpaw,r_fighter_stance_Switch,r_fighter_stance_unlisted,b_fighter_stance_Open Stance,b_fighter_stance_Orthodox,b_fighter_stance_Sideways,b_fighter_stance_Southpaw,b_fighter_stance_Switch,b_fighter_stance_unlisted
0,Red,TRUE,Catch Weight,1,Southpaw,Orthodox,195.58,185.42,184.512378,184.798074,216.0,175.0,34.000000,26.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,182.880000,184.798074,205.0,175.0,29.000000,26.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,184.512378,184.798074,196.0,175.0,30.000000,26.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,Red,FALSE,Open Weight,1,Southpaw,unlisted,187.96,180.34,184.512378,184.798074,185.0,190.0,29.179944,24.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,Red,FALSE,Open Weight,1,Orthodox,Orthodox,193.04,195.58,184.512378,184.798074,275.0,216.0,29.179944,34.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4824,Blue,FALSE,Featherweight,3,Orthodox,Orthodox,180.34,172.72,182.880000,180.340000,145.0,145.0,31.000000,37.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4825,Red,FALSE,Bantamweight,3,Orthodox,Orthodox,167.64,170.18,165.100000,180.340000,135.0,135.0,32.000000,29.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4826,Red,TRUE,Bantamweight,5,Orthodox,Orthodox,167.64,162.56,170.180000,162.560000,135.0,135.0,31.000000,32.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4827,Blue,FALSE,Bantamweight,3,Orthodox,Switch,170.18,162.56,170.180000,172.720000,135.0,135.0,26.000000,29.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [23]:
fnf_df_num_2 = fnf_df_num.copy()

In [24]:
fnf_df_num_2 = fnf_df_num_2.replace(to_replace ="Red ", value = 1.0)
fnf_df_num_2 = fnf_df_num_2.replace(to_replace ="Blue", value = 2.0)
fnf_df_num_2 = fnf_df_num_2.replace(to_replace ="Draw", value = 3.0)

fnf_df_num_2.head(40)

,winner,title_bout,weight_class,no_of_rounds,r_fighter_stance,b_fighter_stance,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight,b_fighter_age,r_fighter_age,title_bout_FALSE,title_bout_TRUE,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,r_fighter_stance_Open Stance,r_fighter_stance_Orthodox,r_fighter_stance_Sideways,r_fighter_stance_Southpaw,r_fighter_stance_Switch,r_fighter_stance_unlisted,b_fighter_stance_Open Stance,b_fighter_stance_Orthodox,b_fighter_stance_Sideways,b_fighter_stance_Southpaw,b_fighter_stance_Switch,b_fighter_stance_unlisted
0,1.0,TRUE,Catch Weight,1,Southpaw,Orthodox,195.580000,185.420000,184.512378,184.798074,216.000000,175.000000,34.000000,26.00000,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,FALSE,Open Weight,1,Southpaw,Orthodox,185.420000,185.420000,182.880000,184.798074,205.000000,175.000000,29.000000,26.00000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,FALSE,Open Weight,1,Southpaw,Orthodox,185.420000,185.420000,184.512378,184.798074,196.000000,175.000000,30.000000,26.00000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,FALSE,Open Weight,1,Southpaw,unlisted,187.960000,180.340000,184.512378,184.798074,185.000000,190.000000,29.179944,24.00000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,FALSE,Open Weight,1,Orthodox,Orthodox,193.040000,195.580000,184.512378,184.798074,275.000000,216.000000,29.179944,34.00000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5,1.0,FALSE,Open Weight,1,Orthodox,Orthodox,182.880000,195.580000,184.512378,184.798074,430.000000,216.000000,24.000000,34.00000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
6,1.0,FALSE,Open Weight,1,Orthodox,Orthodox,187.960000,185.420000,184.512378,182.880000,225.000000,205.000000,30.000000,29.00000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7,1.0,FALSE,Open Weight,1,Orthodox,Orthodox,195.580000,193.040000,184.512378,184.798074,250.000000,275.000000,29.179944,29.46191,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,1.0,FALSE,Open Weight,1,Southpaw,Southpaw,190.500000,185.420000,184.512378,184.798074,260.000000,175.000000,29.179944,27.00000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
9,1.0,FALSE,Open Weight,1,Southpaw,Southpaw,180.340000,185.420000,184.512378,184.798074,190.000000,175.000000,24.000000,27.00000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [25]:
fnf_df_num_2.drop(['title_bout', 'weight_class', 'r_fighter_stance', 'b_fighter_stance'], axis = 1 , inplace = True)
fnf_df_num_2

,winner,no_of_rounds,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight,b_fighter_age,r_fighter_age,title_bout_FALSE,title_bout_TRUE,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,r_fighter_stance_Open Stance,r_fighter_stance_Orthodox,r_fighter_stance_Sideways,r_fighter_stance_Southpaw,r_fighter_stance_Switch,r_fighter_stance_unlisted,b_fighter_stance_Open Stance,b_fighter_stance_Orthodox,b_fighter_stance_Sideways,b_fighter_stance_Southpaw,b_fighter_stance_Switch,b_fighter_stance_unlisted
0,1.0,1,195.58,185.42,184.512378,184.798074,216.0,175.0,34.000000,26.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1.0,1,185.42,185.42,182.880000,184.798074,205.0,175.0,29.000000,26.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1.0,1,185.42,185.42,184.512378,184.798074,196.0,175.0,30.000000,26.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1.0,1,187.96,180.34,184.512378,184.798074,185.0,190.0,29.179944,24.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.0,1,193.04,195.58,184.512378,184.798074,275.0,216.0,29.179944,34.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4824,2.0,3,180.34,172.72,182.880000,180.340000,145.0,145.0,31.000000,37.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4825,1.0,3,167.64,170.18,165.100000,180.340000,135.0,135.0,32.000000,29.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4826,1.0,5,167.64,162.56,170.180000,162.560000,135.0,135.0,31.000000,32.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4827,2.0,3,170.18,162.56,170.180000,172.720000,135.0,135.0,26.000000,29.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [26]:
fnf_df_num_2.dtypes

winner                            float64
no_of_rounds                        int64
b_fighter_height                  float64
r_fighter_height                  float64
b_fighter_reach                   float64
r_fighter_reach                   float64
b_fighter_weight                  float64
r_fighter_weight                  float64
b_fighter_age                     float64
r_fighter_age                     float64
title_bout_FALSE                  float64
title_bout_TRUE                   float64
weight_class_Bantamweight         float64
weight_class_Catch Weight         float64
weight_class_Featherweight        float64
weight_class_Flyweight            float64
weight_class_Heavyweight          float64
weight_class_Light Heavyweight    float64
weight_class_Lightweight          float64
weight_class_Middleweight         float64
weight_class_Open Weight          float64
weight_class_Welterweight         float64
r_fighter_stance_Open Stance      float64
r_fighter_stance_Orthodox         

In [27]:
# Preprocess data, split the data into Training and Testing
    # Create feature set 
X = fnf_df_num_2.copy()
X = X.drop(columns='winner', axis=1)
X

,no_of_rounds,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight,b_fighter_age,r_fighter_age,title_bout_FALSE,title_bout_TRUE,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,r_fighter_stance_Open Stance,r_fighter_stance_Orthodox,r_fighter_stance_Sideways,r_fighter_stance_Southpaw,r_fighter_stance_Switch,r_fighter_stance_unlisted,b_fighter_stance_Open Stance,b_fighter_stance_Orthodox,b_fighter_stance_Sideways,b_fighter_stance_Southpaw,b_fighter_stance_Switch,b_fighter_stance_unlisted
0,1,195.58,185.42,184.512378,184.798074,216.0,175.0,34.000000,26.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,1,185.42,185.42,182.880000,184.798074,205.0,175.0,29.000000,26.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,185.42,185.42,184.512378,184.798074,196.0,175.0,30.000000,26.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,1,187.96,180.34,184.512378,184.798074,185.0,190.0,29.179944,24.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1,193.04,195.58,184.512378,184.798074,275.0,216.0,29.179944,34.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4824,3,180.34,172.72,182.880000,180.340000,145.0,145.0,31.000000,37.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4825,3,167.64,170.18,165.100000,180.340000,135.0,135.0,32.000000,29.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4826,5,167.64,162.56,170.180000,162.560000,135.0,135.0,31.000000,32.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4827,3,170.18,162.56,170.180000,172.720000,135.0,135.0,26.000000,29.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [28]:
# Create target
y = fnf_df_num_2["winner"]
y

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
4824    2.0
4825    1.0
4826    1.0
4827    2.0
4828    2.0
Name: winner, Length: 4829, dtype: float64

In [29]:
# Check the balance of our target values
y.value_counts()

1.0    3288
2.0    1462
3.0      79
Name: winner, dtype: int64

In [30]:
# Split into Train and Test sets. 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [31]:
# Determine the shape of our training and testing sets.
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(3863, 33)
(966, 33)
(3863,)
(966,)


In [32]:
# Create a StandardScaler instance. 
scaler = StandardScaler()

In [33]:
# Fit the Standard Scaler with the training data. 
X_scaler = scaler.fit(X_train)

In [34]:
# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [35]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [36]:
y_pred = brf.predict(X_test)

In [37]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.4635689759390475

In [38]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[288, 157, 214],
       [ 73, 112, 108],
       [  1,   5,   8]])

In [39]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        1.0       0.80      0.44      0.76      0.56      0.58      0.32       659
        2.0       0.41      0.38      0.76      0.40      0.54      0.28       293
        3.0       0.02      0.57      0.66      0.05      0.61      0.37        14

avg / total       0.67      0.42      0.76      0.51      0.57      0.31       966



In [40]:
X.columns

Index(['no_of_rounds', 'b_fighter_height', 'r_fighter_height',
       'b_fighter_reach', 'r_fighter_reach', 'b_fighter_weight',
       'r_fighter_weight', 'b_fighter_age', 'r_fighter_age',
       'title_bout_FALSE', 'title_bout_TRUE ', 'weight_class_Bantamweight',
       'weight_class_Catch Weight', 'weight_class_Featherweight',
       'weight_class_Flyweight', 'weight_class_Heavyweight',
       'weight_class_Light Heavyweight', 'weight_class_Lightweight',
       'weight_class_Middleweight', 'weight_class_Open Weight',
       'weight_class_Welterweight', 'r_fighter_stance_Open Stance',
       'r_fighter_stance_Orthodox', 'r_fighter_stance_Sideways',
       'r_fighter_stance_Southpaw', 'r_fighter_stance_Switch',
       'r_fighter_stance_unlisted', 'b_fighter_stance_Open Stance',
       'b_fighter_stance_Orthodox', 'b_fighter_stance_Sideways',
       'b_fighter_stance_Southpaw', 'b_fighter_stance_Switch',
       'b_fighter_stance_unlisted'],
      dtype='object')

In [41]:
# List the features sorted in descending order by feature importance
sorted(zip(brf.feature_importances_, X_train.columns), reverse=True)

[(0.12558349765577248, 'r_fighter_age'),
 (0.11613629924503353, 'b_fighter_age'),
 (0.1030063145756483, 'r_fighter_reach'),
 (0.10205232228795906, 'b_fighter_reach'),
 (0.09875332004480139, 'b_fighter_height'),
 (0.08963285813847884, 'r_fighter_height'),
 (0.0636435330534751, 'b_fighter_weight'),
 (0.06126531385344318, 'r_fighter_weight'),
 (0.021416123704362317, 'b_fighter_stance_Orthodox'),
 (0.020948646081388085, 'r_fighter_stance_Orthodox'),
 (0.020314987015818714, 'b_fighter_stance_Southpaw'),
 (0.019293264367570978, 'r_fighter_stance_Southpaw'),
 (0.018638772330975856, 'no_of_rounds'),
 (0.017079467050856893, 'weight_class_Middleweight'),
 (0.016417510903500843, 'weight_class_Welterweight'),
 (0.015431842679388732, 'weight_class_Lightweight'),
 (0.012576564873589949, 'weight_class_Light Heavyweight'),
 (0.011322710422525752, 'weight_class_Featherweight'),
 (0.01021438607717378, 'weight_class_Bantamweight'),
 (0.009853533681774688, 'weight_class_Heavyweight'),
 (0.0077461716231706